In [4]:

from pathlib import Path
import sys

REPO_ROOT = Path('..').resolve()
DEMAND_PROFILES_ROOT = REPO_ROOT / 'Codes' / 'Output' / 'DemandProfiles'
SUMMARY_FILE = REPO_ROOT / 'Codes' / 'Data' / '1R1C1P1S_filtered_filtered.csv'
OUTPUT_FILE = REPO_ROOT / 'Codes' / 'Output' /'DemandProfiles'/ 'demand_metrics_summary.csv'

if str(REPO_ROOT) not in sys.path:
    sys.path.append(str(REPO_ROOT))

REPO_ROOT, DEMAND_PROFILES_ROOT, SUMMARY_FILE, OUTPUT_FILE

(WindowsPath('E:/GitHubProjects/LV network'),
 WindowsPath('E:/GitHubProjects/LV network/Codes/Output/DemandProfiles'),
 WindowsPath('E:/GitHubProjects/LV network/Codes/Data/1R1C1P1S_filtered_filtered.csv'),
 WindowsPath('E:/GitHubProjects/LV network/Codes/Output/DemandProfiles/demand_metrics_summary.csv'))

In [5]:
import pandas as pd

from Codes.sourcecode.generate_demand_metrics import compute_metrics

metrics = compute_metrics(
    demand_profiles_root=DEMAND_PROFILES_ROOT,
    summary_path=SUMMARY_FILE,
)
metrics.head()

,Dwelling ID,Device (HHP/mHP+capacity),Tariff Type,Weather (mild/extreme),Peak Electricity Consumption (Feb 11 1600-1900) [kWh],Total Electricity Consumption (Feb10-12) [kWh],Total Gas Consumption (Feb10-12) [kWh],R (K/W),C (J/K),g (m^2)
0,EOH0003,HHP 4.00 kW,Time-of-use,mild,0.0,31.064,0.000,0.007548,4.091954e+07,13.439900
1,EOH0014,HHP 4.00 kW,Time-of-use,mild,0.0,49.187,4.439,0.006439,1.818650e+07,0.249881
2,EOH0018,HHP 4.00 kW,Time-of-use,mild,0.0,57.359,10.818,0.005212,2.267737e+07,2.288106
3,EOH0021,HHP 4.00 kW,Time-of-use,mild,0.0,65.622,38.559,0.004268,9.130042e+07,0.000001
4,EOH0025,HHP 4.00 kW,Time-of-use,mild,0.0,18.655,0.000,0.017066,3.422080e+07,0.000001


In [6]:

# Save to CSV if desired
metrics.to_csv(OUTPUT_FILE, index=False)
print(f'Saved {len(metrics)} rows to {OUTPUT_FILE}')

Saved 4064 rows to E:\GitHubProjects\LV network\Codes\Output\DemandProfiles\demand_metrics_summary.csv


In [8]:
metrics

,Dwelling ID,Device (HHP/mHP+capacity),Tariff Type,Weather (mild/extreme),Peak Electricity Consumption (Feb 11 1600-1900) [kWh],Total Electricity Consumption (Feb10-12) [kWh],Total Gas Consumption (Feb10-12) [kWh],R (K/W),C (J/K),g (m^2)
0,EOH0003,HHP 4.00 kW,Time-of-use,mild,0.000,31.064,0.000,0.007548,4.091954e+07,13.439900
1,EOH0014,HHP 4.00 kW,Time-of-use,mild,0.000,49.187,4.439,0.006439,1.818650e+07,0.249881
2,EOH0018,HHP 4.00 kW,Time-of-use,mild,0.000,57.359,10.818,0.005212,2.267737e+07,2.288106
3,EOH0021,HHP 4.00 kW,Time-of-use,mild,0.000,65.622,38.559,0.004268,9.130042e+07,0.000001
4,EOH0025,HHP 4.00 kW,Time-of-use,mild,0.000,18.655,0.000,0.017066,3.422080e+07,0.000001
...,...,...,...,...,...,...,...,...,...,...
4059,EOH3183,mHP 7.00 kW,Flat,extreme,2.905,30.583,0.000,0.012392,2.410693e+07,2.869670
4060,EOH3186,mHP 8.50 kW,Flat,extreme,6.683,151.297,0.000,0.002672,9.716478e+07,1.365901
4061,EOH3188,mHP 7.00 kW,Flat,extreme,4.089,69.910,0.000,0.005384,1.884267e+07,5.504479
4062,EOH3196,mHP 7.00 kW,Flat,extreme,4.241,75.305,0.000,0.005005,2.778700e+07,7.022169


In [7]:
from Codes.sourcecode.analyze_peak_reduction import (
    compute_peak_reduction,
    plot_histograms,
    plot_scatter_matrix,
)

peak_reduction = compute_peak_reduction(metrics)
peak_reduction.to_csv(
    DEMAND_PROFILES_ROOT / 'peak_demand_reduction_extreme.csv', index=False
)

hist_paths = plot_histograms(peak_reduction, DEMAND_PROFILES_ROOT / 'figures')
scatter_path = plot_scatter_matrix(
    peak_reduction,
    DEMAND_PROFILES_ROOT / 'figures' / 'peak_demand_reduction_scatter.png',
)

print('Saved histogram(s):')
for tech, path in hist_paths.items():
    print(f' - {tech}: {path}')
print(f'Scatter matrix: {scatter_path}')

peak_reduction.head()

Saved histogram(s):
Scatter matrix: E:\GitHubProjects\LV network\Codes\Output\DemandProfiles\figures\peak_demand_reduction_scatter.png


,Dwelling ID,Technology,Capacity kW,R (K/W),C (J/K),g (m^2),Flat,Time-of-use,Peak Demand Reduction (Flat -> Time-of-use) [kWh],Thermal Constant (R*C)
